In [1]:
from data_split_utils import cross_validation_splits

In [ ]:
### implementation of cross_validation
def cross_validation(data, hyperparam_dict, num_folds, site_var_name = 'site'):
    cross_validation_splits(data, num_folds, site_var_name='site')
    
    test_r2_list = []
    mean_r2_list = []
    for penalty in penalty_list:
        
        test_mse_list = []
        for train_ind, test_ind in data_splitter.split(x, y):
            
            train_x = x.iloc[train_ind, :]
            train_y = y.iloc[train_ind]

            test_x = x.iloc[test_ind, :]
            test_y = y.iloc[test_ind]
            
            median_imputer = sklearn.preprocessing.Imputer(strategy = 'median')
            train_x_imp = median_imputer.fit_transform(train_x)
            test_x_imp = median_imputer.transform(test_x)
            
            if interactions == True and poly_degree == 1:
                feature_generator = sklearn.preprocessing.PolynomialFeatures(interaction_only = True, include_bias = False)
                train_x_imp = feature_generator.fit_transform(train_x_imp)
                test_x_imp = feature_generator.transform(test_x_imp)
            elif interactions == True and poly_degree > 1:
                feature_generator = sklearn.preprocessing.PolynomialFeatures(degree = poly_degree, include_bias = False)
                train_x_imp = feature_generator.fit_transform(train_x_imp)
                test_x_imp = feature_generator.transform(test_x_imp)

            standardizer = sklearn.preprocessing.StandardScaler(with_mean = True, with_std = True)
            train_x_imp_std = standardizer.fit_transform(train_x_imp)
            test_x_imp_std = standardizer.transform(test_x_imp)
            
            lasso = sklearn.linear_model.Lasso(alpha=penalty, fit_intercept=True, random_state=1, max_iter = 10000)
            lasso.fit(train_x_imp_std, train_y)
            lasso_test_pred = lasso.predict(test_x_imp_std)
            lasso_test_mse = sklearn.metrics.mean_squared_error(lasso_test_pred, test_y)
            
            test_mse_list.append(lasso_test_mse)
            
        mean_mse = np.mean(test_mse_list)
        mean_mse_list.append(mean_mse)
        
        if penalty == penalty_list[0]:
            best_mean_mse = np.mean(test_mse_list)
            best_penalty = penalty
            
        elif mean_mse < best_mean_mse:
            best_mean_mse = np.mean(test_mse_list)
            best_penalty = penalty
            
    return best_mean_mse, best_penalty, mean_mse_list